In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeRegressor as dtr
from sklearn import tree
from sklearn.model_selection import GridSearchCV, train_test_split
from math import floor
import random
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error
from time import time
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import pandas as pd
from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

print("loading data")
train_data = pd.read_table("../input/train.tsv")
test_data= pd.read_table("../input/test.tsv")
print(train_data.shape)
df_tr = train_data[:200000]
df_te = test_data[:200000]

print(df_tr.shape)
print(df_te.shape)


In [ ]:
'''df_tr = pd.read_table("../input/train.tsv")
df_te = pd.read_table("../input/test.tsv")


print(df_tr.shape)
print(df_te.shape)'''

In [ ]:
df_tr = df_tr.reset_index(drop=True)
df_te = df_te.reset_index(drop=True)

In [ ]:
# check for nulls
print(df_tr.isnull().sum())
print("*"*100)
print(df_te.isnull().sum())

In [ ]:
#Impute values in place of missig values.
df_tr['category_name'] = df_tr['category_name'].fillna("Unknown")
df_te['category_name'] = df_te['category_name'].fillna("Unknown")
df_tr['brand_name'] = df_tr['brand_name'].fillna("Unknown")
df_te['brand_name'] = df_te['brand_name'].fillna("Unknown")
df_tr['item_description'] = df_tr['item_description'].fillna("Unknown")
df_te['item_description'] = df_te['item_description'].fillna("Unknown")

In [ ]:
df_tr.dtypes

In [ ]:
df_tr.price.describe()

In [ ]:
df_tr.columns
print(df_tr.isnull().sum())
print("*"*100)
print(df_te.isnull().sum())

In [ ]:
df_tr.head()

**Train Data Feature Engineering**

In [ ]:
#converting categorical features in train data to meaningful quantitative features
df_train = df_tr.join(df_tr['category_name'].str.split('/', expand=True).rename(columns={0:'category_1' , 1:'category_2', 2: 'category_3'}))
df_train.head()
#df_train.columns

In [ ]:
df_train['cat1_avg'] = df_train['price'].groupby(df_train['category_1']).transform('mean')
df_train.cat1_avg = df_train.cat1_avg.round(1)
#df['cat1_rank'] = df['cat1_avg'].rank(ascending=1)
df_train['cat1_rank'] =  df_train.cat1_avg.rank(method='dense')
#df1 = df[df['category_1']=='Vintage & Collectibles']
#df1
df_train['cat2_avg'] = df_train['price'].groupby(df_train['category_2']).transform('mean')
df_train.cat2_avg = df_train.cat2_avg.round(1)
df_train['cat2_rank'] =  df_train.cat2_avg.rank(method='dense')

df_train['cat3_avg'] = df_train['price'].groupby(df_train['category_3']).transform('mean')
df_train.cat3_avg = df_train.cat3_avg.round(1)
df_train['cat3_rank'] =  df_train.cat3_avg.rank(method='dense')

df_train['brand_avg'] = df_train['price'].groupby(df_train['brand_name']).transform('mean')
df_train.brand_avg = df_train.brand_avg.round(1)
df_train['brand_rank'] =  df_train.brand_avg.rank(method='dense')

In [ ]:
#df_train = df_train.drop([ 'category_1', 'category_2', 'category_3', 3, 4,'cat1_avg', 'cat2_avg', 'cat3_avg', 'brand_avg' ], axis=1)
#df_train = df_train.drop([ 'brand_name', 'category_name',], axis=1)
df_train = df_train[['train_id','name', 'item_condition_id', 'price', 'shipping',  'item_description', 'cat1_rank',  'cat2_rank' ,  'cat3_rank' ,  'brand_rank', 'category_1', 'category_2', 'category_3','brand_name']]
df_train.head()

In [ ]:
df_train['cat1_rank'].fillna(0, inplace=True)
df_train['cat2_rank'].fillna(0, inplace=True)
df_train['cat3_rank'].fillna(0, inplace=True)
df_train['brand_rank'].fillna(0, inplace=True)
df_train['cat1_rank'] = df_train['cat1_rank'].astype(int)
df_train['cat2_rank'] = df_train['cat2_rank'].astype(int)
df_train['cat3_rank'] = df_train['cat3_rank'].astype(int)
df_train['brand_rank'] = df_train['brand_rank'].astype(int)
df_train.head()

In [ ]:
#converting categorical features in test data to meaningful quantitative features
df_test = df_te.join(df_te['category_name'].str.split('/', expand=True).rename(columns={0:'category_1' , 1:'category_2', 2: 'category_3'}))
df_test.head()
#df_test.columns

In [ ]:
df_test['cat1_avg'] = df_train['price'].groupby(df_train['category_1']).transform('mean')
df_test.cat1_avg = df_test.cat1_avg.round(1)
#df['cat1_rank'] = df['cat1_avg'].rank(ascending=1)
df_test['cat1_rank'] =  df_test.cat1_avg.rank(method='dense')
#df1 = df[df['category_1']=='Vintage & Collectibles']
#df1
df_test['cat2_avg'] = df_train['price'].groupby(df_train['category_2']).transform('mean')
df_test.cat2_avg = df_test.cat2_avg.round(1)
df_test['cat2_rank'] =  df_test.cat2_avg.rank(method='dense')

df_test['cat3_avg'] = df_train['price'].groupby(df_train['category_3']).transform('mean')
df_test.cat3_avg = df_test.cat3_avg.round(1)
df_test['cat3_rank'] =  df_test.cat3_avg.rank(method='dense')

df_test['brand_avg'] = df_train['price'].groupby(df_train['brand_name']).transform('mean')
df_test.brand_avg = df_test.brand_avg.round(1)
df_test['brand_rank'] =  df_test.brand_avg.rank(method='dense')

In [ ]:
df_test = df_test[['test_id','name', 'item_condition_id', 'shipping',  'item_description', 'cat1_rank',  'cat2_rank' ,  'cat3_rank' ,  'brand_rank']]
df_test.head()

In [ ]:
df_test['cat1_rank'].fillna(0, inplace=True)
df_test['cat2_rank'].fillna(0, inplace=True)
df_test['cat3_rank'].fillna(0, inplace=True)
df_test['brand_rank'].fillna(0, inplace=True)
df_test['cat1_rank'] = df_test['cat1_rank'].astype(int)
df_test['cat2_rank'] = df_test['cat2_rank'].astype(int)
df_test['cat3_rank'] = df_test['cat3_rank'].astype(int)
df_test['brand_rank'] = df_test['brand_rank'].astype(int)
df_test.head()

In [ ]:
df_train = df_train[['train_id','name', 'item_condition_id', 'price', 'shipping',  'item_description', 'cat1_rank',  'cat2_rank' ,  'cat3_rank' ,  'brand_rank']]
df_train.head()

In [ ]:
#STEMMING we have 1. porter, 2. snowball, 3. wordnetlemmatizer. Lets use wordNetLemmatizer which is not exactly as stemming, 
#the lemmatizer needs the POS(part of speech) of the word, ‘v’ (for verb), to work, and even though it is much slower than the others, 
#it returns an actual english word. 
import nltk.stem
#from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

#Tokenize
import re
def tokenize(text):
    stemmer = PorterStemmer()
    #lemmatizer = WordNetLemmatizer()
    tokens = []
    stems = []
    for row in text:
        rtext = re.sub("[^a-zA-Z]"," ", row)
        token = nltk.word_tokenize(rtext.lower())
        tokens.append(token)
    #print(tokens)
    for i in range(len(tokens)):
        for j in range(len(tokens[i])):
            stems.append(stemmer.stem(tokens[i][j]))
            #stems.append(lemmatizer.lemmatize(tokens[i][j], pos="n")
    return stems  

In [ ]:
t0=time()
train_name = tokenize(df_train['name'])
train_item_ds = tokenize(df_train['item_description'])
print(time()-t0)

In [ ]:
t0=time()
test_name = tokenize(df_test['name'])
test_item_ds = tokenize(df_test['item_description'])
print(time()-t0)

In [ ]:
t0=time()
full_n = train_name + test_name
full_it = train_item_ds + test_item_ds
print(time()-t0)

In [ ]:
t0=time()
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(stop_words='english',  ngram_range=(1,1))
tfidf_full_it = tfidf_vec.fit_transform(full_it)
train_tfidf = tfidf_vec.transform(df_train['item_description'])
test_tfidf = tfidf_vec.transform(df_test['item_description'])

n_comp = 20
svd_obj = TruncatedSVD(n_components=n_comp)#, algorithm='arpack')
svd_obj.fit(tfidf_full_it)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_item_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_item_'+str(i) for i in range(n_comp)]

df_train= pd.concat([df_train, train_svd], axis=1)
df_test = pd.concat([df_test , test_svd], axis=1)
print(time()-t0)


In [ ]:
t1=time()
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,1))
tfidf_full_n = tfidf_vec.fit_transform(full_n)
train_tfidf = tfidf_vec.transform(df_train['name'])
test_tfidf = tfidf_vec.transform(df_test['name'])

n_comp = 20
svd_obj = TruncatedSVD(n_components=n_comp)#, algorithm='arpack')
svd_obj.fit(tfidf_full_n)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_name_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_name_'+str(i) for i in range(n_comp)]
df_train = pd.concat([df_train, train_svd], axis=1)
df_test = pd.concat([df_test, test_svd], axis=1)
print(time()-t1)

In [ ]:
df_train = df_train.drop([ 'name', 'train_id','item_description'], axis=1)
df_train.head()
df_train.shape

In [ ]:
df_test = df_test.drop([ 'name', 'item_description'], axis=1)
df_test.head()

In [ ]:
df_test_f = df_test.drop([ 'test_id'], axis=1)

**Train, validation data set split**

In [ ]:
import numpy as np
df_train["price"] = df_train["price"].astype(int)
y = df_train["price"]


In [ ]:
X= df_train.drop(['price'], axis = 1)
X_train,X_val,Y_train,Y_val = train_test_split(X,y,test_size = 0.2)

X_train.shape

> **Train models**

In [ ]:
t0=time()
#Apply Random forest regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
rmodel = RandomForestRegressor(n_estimators=1000, n_jobs=-1, max_features="sqrt")
rmodel.fit(X_train,Y_train)
print(time()-t0)

In [ ]:
# Apply the Regressor we trained to the valuation data
ypred = rmodel.predict(X_val)
#print('MAE', mean_absolute_error(Y_val, ypred))
print('MSE', mean_squared_error(Y_val, ypred))
#print('R2', r2_score(Y_val, ypred, multioutput='variance_weighted'))
print('M_log_error', mean_squared_log_error(Y_val, ypred))
print('Variance score: %.2f' % r2_score(Y_val, ypred))

In [ ]:
#Apply Linear Regression
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
lregr = linear_model.LinearRegression()

In [ ]:
# Train the model using the training sets
lregr.fit(X_train, Y_train)

In [ ]:
y_predl = lregr.predict(X_val)

In [ ]:
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(Y_val, y_predl))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_val, y_predl))
# Mean square error is high and same as RandomForest

In [ ]:
t0=time()
#Apply stochastic gradient Descent Regression
#Scale the data first as SGD is sensitive to scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)  # Don't cheat - fit only on training data
X_train1 = scaler.transform(X_train)
#Y_train1 = scaler.transform(Y_train)
X_val1 = scaler.transform(X_val)  # apply same transformation to validation data
print(time()-t0)

In [ ]:

from sklearn import linear_model
sgmodel = linear_model.SGDRegressor()
sgmodel.fit(X_train1, Y_train)

In [ ]:
#from sklearn.metrics import score
#print(score(X_train, Y_train))
y_predsg = sgmodel.predict(X_val1)

In [ ]:
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(Y_val, y_predsg))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_val, y_predsg))
# Mean square error is high and same as RandomForest

In [ ]:
# create Deep Learning model
dmodel = Sequential()
dmodel.add(Dense(45, input_dim=46, kernel_initializer='normal', activation='relu'))
dmodel.add(Dense(32, kernel_initializer='normal', activation='relu'))
dmodel.add(Dense(12, kernel_initializer='normal', activation='relu'))
dmodel.add(Dense(1, kernel_initializer='normal'))

In [ ]:
dmodel.summary()
#Deep Learning accuracy was very low (0.002) which is very bad

In [ ]:
# Compile model
dmodel.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
t0=time()
# Fit the model
dmodel.fit(X_train, Y_train, epochs=10)
print(time()-t0)

In [ ]:
# evaluate the model
scores = dmodel.evaluate(X_train, Y_train)
print("\n%s: %.2f%%" % (dmodel.metrics_names[1], scores[1]*100))

In [ ]:
# calculate predictions
#predictions = dmodel.predict(X_val)
# round predictions
#rounded = [round(x[0]) for x in predictions]
#print(rounded)

**Apply model to test data**

In [ ]:
# Apply the Regressor we trained to the test data (which, remember, it has never seen before)
ypred_final = rmodel.predict(df_test_f)

In [ ]:
yfinal = pd.DataFrame({'price': ypred_final })
yfinal['test_id'] = df_test['test_id']

yfinal.to_csv("sample_submission.csv", index=False)
#print(check_output(["ls", "../input"]).decode("utf8"))